In [2]:
import math
import numpy as np
import tensorflow as tf

2023-04-27 10:21:18.650200: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-27 10:21:18.837477: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-27 10:21:18.839002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 10:21:20.402244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:

x_in = np.array([[
  [[0], [0], [0], [0], [0]],
  [[0], [0], [0], [0], [0]],
  [[1], [0], [1], [0], [0]],
  [[0], [0], [0], [0], [0]],
  [[0], [0], [0], [0], [0]], ]])
kernel_in = np.array([
 [ [0,0],[1,0],[0,0] ],
 [ [1,0],[0,1],[1,0] ],
 [ [0,0],[1,0],[0,0] ] ]).reshape((3, 3, 1, 2)) / 4
x = tf.constant(x_in, dtype=tf.float32)
kernel = tf.constant(kernel_in, dtype=tf.float32)
res = tf.nn.conv2d(x, kernel, strides=[1, 1, 1, 1], padding='VALID', data_format='NHWC')
#res.numpy().reshape((3,3))
print(res.numpy().shape)
res.numpy()

(1, 3, 3, 2)


array([[[[0.  , 0.  ],
         [0.25, 0.  ],
         [0.  , 0.  ]],

        [[0.5 , 0.  ],
         [0.  , 0.25],
         [0.25, 0.  ]],

        [[0.  , 0.  ],
         [0.25, 0.  ],
         [0.  , 0.  ]]]], dtype=float32)

In [9]:
box_size = 49
rad = np.zeros((box_size, box_size), dtype=np.float32)
center = box_size // 2
for y in range(box_size):
    for x in range(box_size):
        rad[y,x] = math.sqrt((x - center) * (x - center) + (y - center) * (y - center))
prev_r = -1.0
r_values = [1.2, 2.2, 2.9, 3.7, 4.5]
masks = []
for i in range(box_size):
    r = r_values[i] if i < len(r_values) else prev_r + (1 + prev_r / 12)
    masks.append(np.logical_and(rad > prev_r, rad <= r))
    prev_r = r
    if r > center:
        break
masks.append(rad > prev_r)
print(len(masks))
index_array = np.zeros((box_size, box_size), dtype=np.uint8)
for i in range(len(masks)):
    index_array[masks[i]] = i
index_array
c_code = '{\n'
def str2(x):
    x = ' ' + str(x)
    return x[-2:]
for y in range(box_size):
    c_code += '    { ' + ','.join(str2(x) for x in index_array[y]) + ' },\n'
c_code += '}\n'
print(c_code)


16
{
    { 15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,14,14,14,14,14,14,14,14,14,14,14,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15 },
    { 15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15 },
    { 15,15,15,15,15,15,15,15,15,15,15,15,15,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,15,15,15,15,15,15,15,15,15,15,15,15,15 },
    { 15,15,15,15,15,15,15,15,15,15,15,14,14,14,14,14,14,14,13,13,13,13,13,13,13,13,13,13,13,13,13,14,14,14,14,14,14,14,15,15,15,15,15,15,15,15,15,15,15 },
    { 15,15,15,15,15,15,15,15,15,15,14,14,14,14,14,14,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,14,14,14,14,14,14,15,15,15,15,15,15,15,15,15,15 },
    { 15,15,15,15,15,15,15,15,15,14,14,14,14,14,13,13,13,13,13,13,13,12,12,12,12,12,12,12,13,13,13,13,13,13,13,14,14,14,14,14,15,15,15,15,15,15,15,15,15 },
    { 15,15,15,15,15,15,15,15,14,14,14,14,13,13,13,13,13,13